In [ ]:
from __future__ import annotations

import sys
sys.path.insert(0, "..")
from src.config import PROJECT_ROOT, LABELS

from pathlib import Path
import glob
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, roc_auc_score


In [ ]:
# =====================
# 0) Paths (EDIT HERE)
# =====================
BASE_DIR = Path(
    str(PROJECT_ROOT / "results/new_analysis/modeling/step2_modeling/simple_20")
)
OUT_DIR = BASE_DIR / "figures/Fig2_ROC_Youden"
OUT_DIR.mkdir(parents=True, exist_ok=True)

In [ ]:
# =====================
# 1) Helpers
# =====================
YTRUE_CANDS = ["y_true", "label", "target", "y", "y_test"]
YPROB_CANDS = ["y_prob", "pred_prob", "score", "prob_1", "p1", "y_pred_proba"]

# Best model mapping per outcome
BEST_MODELS = {
    "30d": "Random Forest",
    "60d": "Random Forest",
    "90d": "Random Forest",
    "180d": "Random Forest",
    "365d": "Random Forest",
}

SHOW_ANNOTATION_BOX = False


def _pick_cols(df: pd.DataFrame, path: str):
    yt = next((c for c in YTRUE_CANDS if c in df.columns), None)
    yp = next((c for c in YPROB_CANDS if c in df.columns), None)
    if yt is None or yp is None:
        raise KeyError(
            f"{path}: missing columns. Need one of {YTRUE_CANDS} and one of {YPROB_CANDS}; have {list(df.columns)}"
        )
    return yt, yp


def _find_youden_file(base: Path, outcome: str) -> Path | None:
    # support: __ and ___ before 'youden'
    pats = [
        str(base / f"label_{outcome}__youden_metrics_Test.csv"),
        str(base / f"label_{outcome}___youden_metrics_Test.csv"),
    ]
    for p in pats:
        for g in glob.glob(p):
            fp = Path(g)
            if fp.exists():
                return fp
    return None


def _outcome_from_preds_path(p: Path) -> str:
    m = re.match(r"label_(\w+)__test_predictions\.csv", p.name)
    return m.group(1) if m else p.stem


def _fpr_tpr_at_thr(y_true: np.ndarray, y_prob: np.ndarray, thr: float):
    y_pred = (y_prob >= thr).astype(int)
    tp = int(((y_pred == 1) & (y_true == 1)).sum())
    tn = int(((y_pred == 0) & (y_true == 0)).sum())
    fp = int(((y_pred == 1) & (y_true == 0)).sum())
    fn = int(((y_pred == 0) & (y_true == 1)).sum())
    tpr = tp / (tp + fn) if (tp + fn) else 0.0
    fpr = fp / (fp + tn) if (fp + tn) else 0.0
    return fpr, tpr, tp, tn, fp, fn

In [ ]:
# =====================
# 2) Discover outcomes
# =====================
PRED_FILES = sorted(BASE_DIR.glob("label_*__test_predictions.csv"))
if not PRED_FILES:
    raise FileNotFoundError(
        f"No prediction files found in {BASE_DIR} (pattern label_*__test_predictions.csv)"
    )

In [ ]:
# =====================
# 3) Main loop
# =====================
rows = []
for ppath in PRED_FILES:
    outcome = _outcome_from_preds_path(ppath)
    dfp = pd.read_csv(ppath)
    yt_col, yp_col = _pick_cols(dfp, str(ppath))
    y_true = dfp[yt_col].astype(int).to_numpy()
    y_prob = dfp[yp_col].astype(float).to_numpy()

    # ROC curve + AUC
    fpr, tpr, thr_arr = roc_curve(y_true, y_prob)
    auc = roc_auc_score(y_true, y_prob)

    # Youden (prefer file, else compute)
    yfile = _find_youden_file(BASE_DIR, outcome)
    if yfile is not None:
        dfy = pd.read_csv(yfile)
        rec = dfy.iloc[0]
        thr = float(rec["threshold"])
        sens = float(rec.get("sensitivity", np.nan))
        spec = float(rec.get("specificity", np.nan))
        yfpr, ytpr, tp, tn, fp, fn = _fpr_tpr_at_thr(y_true, y_prob, thr)
        youden_src = "file"
        if np.isnan(sens) or np.isnan(spec):
            sens = ytpr
            spec = 1 - yfpr
    else:
        J = tpr - fpr
        idx = int(np.argmax(J))
        thr = float(thr_arr[idx])
        yfpr, ytpr, tp, tn, fp, fn = _fpr_tpr_at_thr(y_true, y_prob, thr)
        sens, spec = ytpr, 1 - yfpr
        youden_src = "computed"

    best_model = BEST_MODELS.get(outcome, "Best Model")

    # ============
    # Plot figure
    # ============
    plt.rcParams.update(
        {
            "figure.dpi": 120,
            "axes.spines.top": False,
            "axes.spines.right": False,
            "axes.grid": True,
            "grid.alpha": 0.25,
            "grid.linestyle": "--",
            "legend.frameon": False,
        }
    )

    fig = plt.figure(figsize=(6.4, 6.4))
    (line,) = plt.plot(fpr, tpr, linewidth=2)
    plt.plot([0, 1], [0, 1], linestyle=":", linewidth=1)

    # Youden marker
    pt = plt.scatter(
        [yfpr], [ytpr], s=60, marker="o", edgecolor="black", linewidths=0.8
    )

    # Legend: AUC first, then Youden info directly beneath
    auc_label = f"AUC = {auc:.3f}"
    youden_label = f"Youden index (t={thr:.3f}, Se={sens:.3f}, Sp={spec:.3f})"
    plt.legend(handles=[line, pt], labels=[auc_label, youden_label], loc="lower right")

    # Optional annotation box (off by default)
    if SHOW_ANNOTATION_BOX:
        txt = (
            f"Youden threshold = {thr:.3f}\n"
            f"Sensitivity = {sens:.3f}\n"
            f"Specificity = {spec:.3f}"
        )
        x_text = min(0.85, max(0.55, yfpr + 0.12))
        y_text = min(0.35, max(0.15, ytpr - 0.18))
        plt.text(
            x_text,
            y_text,
            txt,
            fontsize=10,
            bbox=dict(
                boxstyle="round,pad=0.3",
                edgecolor="black",
                facecolor="white",
                alpha=0.85,
            ),
        )

    plt.xlim(0, 1)
    plt.ylim(0, 1)
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.title(f"{outcome} ROC curve for Best model ({best_model})")

    out_png = OUT_DIR / f"Fig2_ROC_Youden_{outcome}.png"
    out_pdf = OUT_DIR / f"Fig2_ROC_Youden_{outcome}.pdf"
    fig.savefig(out_png, bbox_inches="tight")
    fig.savefig(out_pdf, bbox_inches="tight")
    plt.close(fig)

    rows.append(
        {
            "outcome": outcome,
            "best_model": best_model,
            "predictions": str(ppath),
            "youden_source": youden_src,
            "youden_file": str(yfile) if yfile else "",
            "threshold": thr,
            "sensitivity": sens,
            "specificity": spec,
            "fpr_at_thr": yfpr,
            "tpr_at_thr": ytpr,
            "auc": auc,
            "png": str(out_png),
            "pdf": str(out_pdf),
            "tp": tp,
            "tn": tn,
            "fp": fp,
            "fn": fn,
            "n_pos": int((y_true == 1).sum()),
            "n_neg": int((y_true == 0).sum()),
            "n": len(y_true),
        }
    )

In [ ]:
# =====================
# 4) Save summary
# =====================
summary_df = pd.DataFrame(rows).sort_values("outcome")
sum_csv = OUT_DIR / "Fig2_ROC_Youden_summary.csv"
summary_df.to_csv(sum_csv, index=False)
print(f"Saved: {sum_csv}")
# Optional: show a quick preview if running interactively
try:
    from caas_jupyter_tools import display_dataframe_to_user

    display_dataframe_to_user("Fig2 ROC + Youden Summary", summary_df)
except Exception:
    pass

# =====================
# (Optional) Multi-panel composer
# =====================
# If you prefer a single multi-panel figure (e.g., 2x3 for 5 outcomes),
# you can stitch saved PNGs using PIL. Uncomment below.

from PIL import Image
import math

# 이미지 로드
imgs = [Image.open(p) for p in summary_df["png"]]
n = len(imgs)
cols = 3
rows = 2  # 고정 2행 구성 (위 3개, 아래 2개)
w, h = imgs[0].size

# 새 캔버스 (흰 배경)
sheet = Image.new("RGB", (cols * w, rows * h), "white")

# 원하는 outcome 순서로 정렬 (확실하게 지정)
order = ["30d", "60d", "90d", "180d", "365d"]
imgs_ordered = []
for o in order:
    # summary_df 내 outcome 컬럼에서 해당 이미지 경로 찾기
    path = summary_df.loc[summary_df["outcome"] == o, "png"].values
    if len(path):
        imgs_ordered.append(Image.open(path[0]))

# 위 행(0): 3개, 아래 행(1): 2개 가운데 정렬
# --- 첫 행: 0,1,2열에 30d·60d·90d
for i in range(3):
    sheet.paste(imgs_ordered[i], (i * w, 0))

# --- 둘째 행: 가운데 정렬 → 열 0부터 시작하지 않고 0.5칸 띄워서
#     (즉, 180d는 col=0.5, 365d는 col=1.5)
x_offset_left = w // 2  # 한 칸의 절반만큼 좌측 여백
sheet.paste(imgs_ordered[3], (x_offset_left, h))
sheet.paste(imgs_ordered[4], (x_offset_left + w, h))

# 저장
sheet_path = OUT_DIR / "Fig2_ROC_Youden_COMPOSITE.png"
sheet.save(sheet_path)
print("Saved composite:", sheet_path)

# SHAP summary plot

In [ ]:
import os
from pathlib import Path
from PIL import Image

# ==========================================
# 1. 설정 (경로 및 파일명 패턴)
# ==========================================
# 이미지 파일이 위치한 디렉토리
IMG_DIR = Path(
    str(PROJECT_ROOT / "results/new_analysis/modeling/step2_modeling/simple_20/figures")
)

# 결과물을 저장할 경로 (동일 폴더 내 저장)
OUT_PATH = IMG_DIR / "Total_SHAP_summary_COMPOSITE.png"

# 통합할 Outcome 순서
outcomes = ["label_30d", "label_60d", "label_90d", "label_180d", "label_365d"]

# 개별 파일명 패턴 (예: 30d__SHAP_summary_Test.png)
file_pattern = "{}__SHAP_summary_Test.png"

# ==========================================
# 2. 이미지 로드
# ==========================================
imgs = []
print("--- Loading Images ---")
for outcome in outcomes:
    fname = file_pattern.format(outcome)
    fpath = IMG_DIR / fname

    if fpath.exists():
        img = Image.open(fpath)
        imgs.append(img)
        print(f"Loaded: {fname}")
    else:
        print(f"[Warning] File not found: {fname}")

# 이미지가 하나도 없으면 종료
if not imgs:
    raise FileNotFoundError("지정된 경로에서 이미지를 찾을 수 없습니다.")

# ==========================================
# 3. 캔버스 생성 및 배치 (3행 2열 중앙 정렬 구조)
# ==========================================
# 기준 사이즈 (첫 번째 이미지 기준)
w, h = imgs[0].size
n_imgs = len(imgs)

# 그리드 설정: 가로 3개, 세로 2줄
cols = 3
rows = 2
canvas_w = cols * w
canvas_h = rows * h

# 흰색 배경 캔버스 생성
sheet = Image.new("RGB", (canvas_w, canvas_h), "white")

# --- 첫째 줄 (Index 0, 1, 2) ---
# 30d, 60d, 90d 배치
for i in range(min(3, n_imgs)):
    sheet.paste(imgs[i], (i * w, 0))

# --- 둘째 줄 (Index 3, 4) ---
# 180d, 365d 배치 (가운데 정렬)
# 로직: 전체 폭이 3w이고 이미지가 2개(2w)이므로, 남은 여백은 w.
# 좌우 여백을 균등하게 w/2씩 주면 됨. 시작점 = w // 2
if n_imgs > 3:
    x_offset_start = w // 2  # 0.5칸 띄우기

    # 180d (Index 3)
    if n_imgs >= 4:
        sheet.paste(imgs[3], (x_offset_start, h))

    # 365d (Index 4)
    if n_imgs >= 5:
        sheet.paste(imgs[4], (x_offset_start + w, h))

# ==========================================
# 4. 저장
# ==========================================
sheet.save(OUT_PATH)
print("-" * 30)
print(f"Successfully saved composite image:\n{OUT_PATH}")

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
import seaborn as sns

os.chdir(
    str(PROJECT_ROOT / "results/new_analysis/modeling/step2_modeling/simple_20")
)

# 1. 파일 목록 정의 (업로드하신 파일명과 매칭)
scenarios = [
    {
        "label": "30 Days",
        "file": "label_30d__test_predictions.csv",
        "color": "#1f77b4",
    },  # 파랑
    {
        "label": "60 Days",
        "file": "label_60d__test_predictions.csv",
        "color": "#ff7f0e",
    },  # 주황
    {
        "label": "90 Days",
        "file": "label_90d__test_predictions.csv",
        "color": "#2ca02c",
    },  # 초록
    {
        "label": "180 Days",
        "file": "label_180d__test_predictions.csv",
        "color": "#d62728",
    },  # 빨강
    {
        "label": "365 Days",
        "file": "label_365d__test_predictions.csv",
        "color": "#9467bd",
    },  # 보라
]

# 2. 그래프 그리기
plt.figure(figsize=(10, 8))
sns.set_style("whitegrid")

for scenario in scenarios:
    try:
        # 파일 읽기
        df = pd.read_csv(scenario["file"])

        # 데이터 추출 (파일명 확인 완료: y_true, y_prob 사용)
        y_true = df["y_true"]
        y_score = df["y_prob"]

        # ROC 곡선 계산
        fpr, tpr, _ = roc_curve(y_true, y_score)
        roc_auc = auc(fpr, tpr)

        # 플롯 추가
        plt.plot(
            fpr,
            tpr,
            color=scenario["color"],
            lw=2,
            label=f"{scenario['label']} (AUC = {roc_auc:.2f})",
        )

    except Exception as e:
        print(f"[Error] {scenario['label']} 처리 중 오류: {e}")

# 3. 디자인 상세 설정
plt.plot(
    [0, 1], [0, 1], color="navy", lw=2, linestyle="--"
)  # 대각선 label='Random Guess'
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel("False Positive Rate (1 - Specificity)", fontsize=25)
plt.ylabel("True Positive Rate (Sensitivity)", fontsize=25)
plt.title("ROC Curves for Prediction Models across All Outcome", fontsize=25, pad=20)
plt.legend(loc="lower right", frameon=True, fontsize=20)
plt.grid(True, alpha=0.5)
plt.tight_layout()

# 그래프 출력
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np

# 1. 데이터 입력 (이미지에서 추출한 값)
data = {
    "Timepoint": ["30d", "60d", "90d", "180d", "365d"] * 2,
    "Feature Set": ["Biomarker (Lab)"] * 5 + ["SDoMH (LLM)"] * 5,
    "Delta AUC (%)": [
        6.68,
        4.62,
        -2.61,
        5.56,
        1.71,  # Biomarker (Lab) Values
        5.57,
        1.75,
        0.85,
        5.52,
        0.24,
    ],  # SDoMH (LLM) Values
    "P-value": [
        0.035,
        0.032,
        0.392,
        0.033,
        0.432,  # Biomarker P-values
        0.009,
        0.441,
        0.703,
        0.020,
        0.871,
    ],  # SDoMH P-values
}

df = pd.DataFrame(data)

# 2. 그래프 스타일 설정 (포스터용 깔끔한 디자인)
sns.set_theme(style="whitegrid", context="talk")  # talk context는 폰트를 크게 만듦
plt.figure(figsize=(10, 6))

# 3. 막대 그래프 그리기
# 색상: Biomarker는 임상적(파랑 계열), SDoMH는 사회적(주황/초록 계열) 느낌
palette = {"Biomarker (Lab)": "#4C72B0", "SDoMH (LLM)": "#55A868"}
ax = sns.barplot(
    data=df,
    x="Timepoint",
    y="Delta AUC (%)",
    hue="Feature Set",
    palette=palette,
    edgecolor="black",  # 테두리 추가로 가독성 향상
    linewidth=1.2,
)

# 4. 유의성(P < 0.05) 표시 (별표 추가)
# 각 막대의 높이와 위치를 계산해서 별표를 찍습니다.
for p, p_val, val in zip(ax.patches, df["P-value"], df["Delta AUC (%)"]):
    if p_val < 0.05:
        # 막대 높이보다 약간 위에 별표 표시
        height = p.get_height()
        ax.text(
            p.get_x() + p.get_width() / 2.0,
            height + 0.3,  # 별표 위치 (Y축) 조정
            "*",
            ha="center",
            va="bottom",
            color="red",
            fontsize=20,
            fontweight="bold",
        )

# 5. 디자인 다듬기 (범례 수정)
plt.axhline(0, color="black", linewidth=1)
plt.title(
    "Performance Improvement over Base Feature", fontsize=20, fontweight="bold", pad=20
)
plt.ylabel("Δ AUC (%)", fontsize=20, fontweight="bold")
plt.xlabel("Timepoint", fontsize=20, fontweight="bold")

# === [수정된 부분] 범례 크기 축소 ===
plt.legend(
    title=None,
    loc="upper right",  # 위치: 오른쪽 상단 (안쪽)
    fontsize=13,  # 폰트 크기: 13 (기본값보다 작게 설정하여 공간 확보)
    frameon=True,  # 테두리 표시
    framealpha=0.9,  # 배경 불투명도 (그래프 눈금 위에 겹쳐도 글자가 잘 보이게)
)

plt.ylim(-4, 9)

# 6. 저장 및 출력
plt.tight_layout()
# plt.savefig("performance_comparison.png", dpi=300) # 파일로 저장하려면 주석 해제
plt.show()

In [ ]:
# 1. 데이터 입력 (새로 제공한 값 기반)
data = {
    "Timepoint": ["30d", "60d", "90d", "180d", "365d"] * 2,
    "Feature Set": ["SDoMH (LLM)"] * 5 + ["Biomarker (Lab)"] * 5,
    "Delta AUC (%)": [
        3.212,   # 30d LLM
        -0.844,  # 60d LLM
        0.264,   # 90d LLM
        -0.914,  # 180d LLM
        0.382,   # 365d LLM
        10.827,  # 30d Lab
        7.331,   # 60d Lab
        -0.145,  # 90d Lab
        4.769,   # 180d Lab
        7.251,   # 365d Lab
    ],
    "P-value": [
        1.70e-01,  # LLM
        8.00e-01,
        8.99e-01,
        6.33e-01,
        8.80e-01,
        1.98e-04,  # Lab
        3.13e-02,
        9.50e-01,
        6.84e-02,
        2.95e-03,
    ],
}

df = pd.DataFrame(data)

# Timepoint 순서 고정 (중요)
df["Timepoint"] = pd.Categorical(
    df["Timepoint"], categories=["30d", "60d", "90d", "180d", "365d"], ordered=True
)

# 2. 그래프 스타일 설정
sns.set_theme(style="whitegrid", context="talk")
plt.figure(figsize=(10, 6))

# 3. 막대 그래프
palette = {"Biomarker (Lab)": "#4C72B0", "SDoMH (LLM)": "#55A868"} 

ax = sns.barplot(
    data=df,
    x="Timepoint",
    y="Delta AUC (%)",
    hue="Feature Set",
    palette=palette,
    edgecolor="black",
    linewidth=1.2,
)

# 4. 유의성 표시 (P < 0.05)
for patch, (_, row) in zip(ax.patches, df.iterrows()):
    if row["P-value"] < 0.05:
        height = patch.get_height()
        ax.text(
            patch.get_x() + patch.get_width() / 2,
            height + 0.1,
            "*",
            ha="center",
            va="bottom",
            color="red",
            fontsize=20,
            fontweight="bold",
        )

# 5. 디자인 마무리
plt.axhline(0, color="black", linewidth=1)
plt.title(
    "Performance Improvement over Base Model",
    fontsize=20,
    fontweight="bold",
    pad=20,
)
plt.ylabel("Δ AUC (%)", fontsize=20, fontweight="bold")
plt.xlabel("Timepoint", fontsize=20, fontweight="bold")

plt.legend(
    title=None,
    loc="upper right",
    fontsize=13,
    frameon=True,
    framealpha=0.9,
)

plt.ylim(-2, 12)

# 6. 출력
plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np

# 1. 데이터 입력 (총 10개 변수 모두 포함)
data = {
    "Feature": [
        "≥2 ER Visits (Util)",  # 1
        "≥3 Admissions (Util)",  # 2
        "AUDIT-Score (Psych)",  # 3
        "Na (Lab)",  # 4
        "Halluc/Delusion/Paranoia (LLM)",  # 5
        "Abuse/Victimization (LLM)",  # 6
        "Creatinine (Lab)",  # 7
        "ASRS-Score (Psych)",  # 8 (New)
        "ALP (Lab)",  # 9 (New)
        "Lack of Support (LLM)",  # 10
    ],
    "Domain": [
        "Utilization",
        "Utilization",
        "Psychometric",
        "Lab",
        "LLM",
        "LLM",
        "Lab",
        "Psychometric",
        "Lab",
        "LLM",
    ],
    "Top Cluster": ["C3", "C4", "C3", "C3", "C2", "C2", "C4", "C4", "C3", "C4"],
    "Bottom Cluster": ["C2", "C3", "C0", "C1", "C3", "C4", "C0", "C1", "C1", "C0"],
    "Delta SHAP": [
        0.240,
        0.144,
        0.024,
        0.019,
        0.018,
        0.017,
        0.016,
        0.016,
        0.015,
        0.013,
    ],
}

df = pd.DataFrame(data)
# 시각적 정렬을 위해 Delta SHAP 오름차순 정렬 (큰 값이 위로 가게)
df = df.sort_values("Delta SHAP", ascending=True)

# 2. 그래프 스타일 설정
sns.set_theme(style="whitegrid", context="talk")
# 변수가 10개로 늘어났으므로 높이를 8로 조금 늘림
fig, ax = plt.subplots(figsize=(11, 8))

# 도메인별 색상 팔레트
domain_palette = {
    "LLM": "#55A868",  # 초록
    "Utilization": "#C44E52",  # 빨강
    "Lab": "#4C72B0",  # 파랑
    "Psychometric": "#8172B3",  # 보라
}
cluster_color = "#CC0000"

# 3. 가로 막대 그래프 그리기
bars = ax.barh(
    y=df["Feature"],
    width=df["Delta SHAP"],
    color=[domain_palette[d] for d in df["Domain"]],
    edgecolor="black",
    linewidth=0.8,
    alpha=0.8,
)

# 4. 클러스터 비교 레이블 추가
for i, bar in enumerate(bars):
    width = bar.get_width()
    y_pos = bar.get_y() + bar.get_height() / 2

    top_c = df.iloc[i]["Top Cluster"]
    bottom_c = df.iloc[i]["Bottom Cluster"]

    # 막대 오른쪽 끝 (Higher in ...)
    ax.text(
        width + 0.005,
        y_pos,
        f" Higher in {top_c}",
        va="center",
        ha="left",
        fontsize=12,
        fontweight="bold",
        color=cluster_color,
    )

    # 막대 왼쪽 끝 (vs ...)
    ax.text(
        -0.005,
        y_pos,
        f"vs {bottom_c} ",
        va="center",
        ha="right",
        fontsize=11,
        color="gray",
        fontstyle="italic",
    )

# 5. 디자인 다듬기
ax.set_xlabel(
    "Difference in Feature Importance (Δ mean SHAP)", fontsize=14, fontweight="bold"
)
ax.set_title(
    "Distinct Risk Drivers by Patient Cluster", fontsize=18, fontweight="bold", pad=20
)

# X축 범위: 글자가 잘리지 않도록 넉넉하게 설정 (-0.06 ~ 0.38)
ax.set_xlim(-0.06, 0.38)
ax.grid(axis="y")

# 범례 추가
from matplotlib.lines import Line2D

legend_elements = [
    Line2D([0], [0], color=color, lw=8, label=domain)
    for domain, color in domain_palette.items()
]
ax.legend(
    handles=legend_elements,
    title="Feature Domain",
    loc="lower right",
    fontsize=11,
    title_fontsize=12,
)

plt.tight_layout()
# plt.savefig("cluster_heterogeneity_chart_all_10.png", dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os

# 1. 파일 경로 및 파일명 설정
base_path = str(PROJECT_ROOT / "results/new_analysis/modeling/step2_modeling/simple_20/figures/clinic_interpretation/PatientClusters")

# 시간 순서대로 정렬 (30d -> 60d -> 90d -> 180d -> 365d)
# 주의: 365d 파일은 확장자가 .png이고 나머지는 .jpg인 점을 반영했습니다.
file_names = [
    "label_30d/label_30d__UMAP2D_DBSCAN_.png",
    "label_60d/label_60d__UMAP2D_DBSCAN_.png",
    "label_90d/label_90d__UMAP2D_DBSCAN_.png",
    "label_180d/label_180d__UMAP2D_DBSCAN_.png",
    "label_365d/label_365d__UMAP2D_DBSCAN_.png",
]

# 2. Figure 생성 (2행 3열 구조)
# figsize는 (가로, 세로) 크기입니다. 필요에 따라 조절하세요.
fig, axes = plt.subplots(2, 3, figsize=(20, 12), constrained_layout=True)

# axes를 1차원 배열로 변환하여 반복문 돌리기 쉽게 만듦
axes_flat = axes.flatten()

# 3. 이미지 로드 및 배치
for i, file_name in enumerate(file_names):
    full_path = os.path.join(base_path, file_name)

    if os.path.exists(full_path):
        img = mpimg.imread(full_path)
        axes_flat[i].imshow(img)
        axes_flat[i].axis("off")  # 축(좌표) 숨기기
        # 필요하다면 아래 주석을 해제하여 서브 타이틀 추가 가능
        # axes_flat[i].set_title(f"Period: {file_name.split('_')[1]}", fontsize=14)
    else:
        print(f"파일을 찾을 수 없습니다: {full_path}")

# 4. 남는 빈 칸(6번째 칸) 처리
for j in range(len(file_names), len(axes_flat)):
    axes_flat[j].axis("off")

# 5. 전체 제목 (선택 사항)
# fig.suptitle('Evolution of Risk Patterns over Time', fontsize=20)

# 6. 결과 저장 및 출력
output_path = os.path.join(base_path, "merged_risk_patterns.png")
# dpi=300은 논문용 고해상도 설정입니다.
plt.savefig(output_path, dpi=300, bbox_inches="tight")

print(f"이미지가 성공적으로 저장되었습니다: {output_path}")
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os

# 1. 파일 경로 설정
base_path = str(PROJECT_ROOT / "results/new_analysis/modeling/step2_modeling/simple_20/figures/clinic_interpretation/PatientClusters")

# 2. 파일명 리스트 (시간 순서대로 정렬: 30d -> 60d -> 90d -> 180d -> 365d)
file_names = [
    "label_30d/label_30d__heatmap_top10_dbscan.png",
    "label_60d/label_60d__heatmap_top10_dbscan.png",
    "label_90d/label_90d__heatmap_top10_dbscan.png",
    "label_180d/label_180d__heatmap_top10_dbscan.png",
    "label_365d/label_365d__heatmap_top10_dbscan.png",
]

# 3. Figure 생성 (2행 3열 구조)
# 히트맵은 가로로 긴 경우가 많으므로 figsize를 조금 더 넓게 조정했습니다.
fig, axes = plt.subplots(2, 3, figsize=(24, 14), constrained_layout=True)

# axes를 1차원 배열로 변환
axes_flat = axes.flatten()

# 4. 이미지 로드 및 배치
for i, file_name in enumerate(file_names):
    full_path = os.path.join(base_path, file_name)

    if os.path.exists(full_path):
        img = mpimg.imread(full_path)
        axes_flat[i].imshow(img)
        axes_flat[i].axis("off")  # 축 숨기기
    else:
        print(f"파일을 찾을 수 없습니다: {full_path}")
        # 파일이 없을 경우 빈 칸 처리
        axes_flat[i].axis("off")

# 5. 남는 빈 칸(6번째 칸) 처리
for j in range(len(file_names), len(axes_flat)):
    axes_flat[j].axis("off")

# 6. 결과 저장
output_path = os.path.join(base_path, "merged_heatmaps.png")
plt.savefig(output_path, dpi=300, bbox_inches="tight")

print(f"이미지가 성공적으로 저장되었습니다: {output_path}")
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os

# 1. 파일 경로 설정
base_path = str(PROJECT_ROOT / "results/new_analysis/modeling/step2_modeling/simple_20/figures/clinic_interpretation/PatientClusters")

# 2. 파일명 리스트 (시간 순서대로 정렬: 30d -> 60d -> 90d -> 180d -> 365d)
file_names = [
    "label_30d/label_30d__heatmap_top10_dbscan__RAW.png",
    "label_60d/label_60d__heatmap_top10_dbscan__RAW.png",
    "label_90d/label_90d__heatmap_top10_dbscan__RAW.png",
    "label_180d/label_180d__heatmap_top10_dbscan__RAW.png",
    "label_365d/label_365d__heatmap_top10_dbscan__RAW.png",
]

# 3. Figure 생성 (2행 3열 구조)
# 히트맵은 가로로 긴 경우가 많으므로 figsize를 조금 더 넓게 조정했습니다.
fig, axes = plt.subplots(2, 3, figsize=(24, 14), constrained_layout=True)

# axes를 1차원 배열로 변환
axes_flat = axes.flatten()

# 4. 이미지 로드 및 배치
for i, file_name in enumerate(file_names):
    full_path = os.path.join(base_path, file_name)

    if os.path.exists(full_path):
        img = mpimg.imread(full_path)
        axes_flat[i].imshow(img)
        axes_flat[i].axis("off")  # 축 숨기기
    else:
        print(f"파일을 찾을 수 없습니다: {full_path}")
        # 파일이 없을 경우 빈 칸 처리
        axes_flat[i].axis("off")

# 5. 남는 빈 칸(6번째 칸) 처리
for j in range(len(file_names), len(axes_flat)):
    axes_flat[j].axis("off")

# 6. 결과 저장
output_path = os.path.join(base_path, "merged_heatmaps__RAW.png")
plt.savefig(output_path, dpi=300, bbox_inches="tight")

print(f"이미지가 성공적으로 저장되었습니다: {output_path}")
plt.show()